In [111]:
import cv2 as cv
import os
import pandas as pd

In [112]:
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)

e:\Hi5-data\ViTPose


In [113]:
root_data_path = ROOT_DIR +'\\temp_data\\'
print(root_data_path)

root_image_path = 'E:\Hi5-data\\train10k_blur'
print(root_image_path)

e:\Hi5-data\ViTPose\temp_data\
E:\Hi5-data\train10k_blur


In [114]:
# check size of all images in the folder
for i in range(1000, 9999):
    #check if the file exists
    if os.path.isfile(root_data_path + '\\00000000' + str(i) + '.png'):
        img = cv.imread(root_data_path + '\\00000000' + str(i) + '.png')
        print(i, img.shape)
        break
    # img = cv.imread(root_data_path + '\\' + str(i) + '.jpg')
    # print(img.shape)

assume all files are of the size: (576, 720, 3)

# CSV to COCO JSON


In [115]:
import json
import csv
def make_json(csv_path, row = None, json_path=None):
    data = {}
    with open(csv_path, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        for rows in csvReader:
            if row == None:
                row = 'ID'
            key = rows[row]
            data[key] = rows

    if json_path is None:
        json_path = csv_path.replace('.csv', 'data.json')
    with open(json_path, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
    
    return data

In [116]:
def make_info_json(description, version, year, date_created):
    info = {}
    info['description'] = description
    info['version'] = version
    info['year'] = year
    info['date_created'] = date_created
    return info


In [117]:
def getFileName(key):
    #all image have 12 digit file name
    key_size = len(key)
    zero_path = '000000000000'
    file_name = zero_path[:12-key_size] + key + '.png'
    if os.path.isfile(root_image_path + '\\' + file_name):
        # print(file_name)
        return file_name
    else:
        print('file not found: ', file_name, 'key: ', key, '12-key_size: ',12-key_size)
        return None

In [118]:
def getKeyPoints(data_json):
    #x,y,confidence
    key_points = [
            # Wrist, dataset 0
            float(data_json['WristX']), float(data_json['WristY']), 1,
             # Thumb, datasset 1-4
            float(data_json['Thumb_A_X_Pos']), float(data_json['Thumb_A_Y_Pos']), 1,
            float(data_json['Thumb_B_X_Pos']), float(data_json['Thumb_B_Y_Pos']), 1,
            float(data_json['Thumb_C_X_Pos']), float(data_json['Thumb_C_Y_Pos']), 1,
            float(data_json['Thumb_D_X_Pos']), float(data_json['Thumb_D_Y_Pos']), 1,
            #Index, dataset 5-8
            float(data_json['Index_A_X_Pos']), float(data_json['Index_A_Y_Pos']), 1,
            float(data_json['Index_B_X_Pos']), float(data_json['Index_B_X_Pos']), 1,
            float(data_json['Index_C_X_Pos']), float(data_json['Index_C_Y_Pos']), 1,
            float(data_json['Index_D_X_Pos']), float(data_json['Index_D_Y_Pos']), 1,
            # Middle, dataset 9-12
            float(data_json['Middle_A_X_Pos']), float(data_json['Middle_A_Y_Pos']), 1,
            float(data_json['Middle_B_X_Pos']), float(data_json['Middle_B_Y_Pos']), 1,
            float(data_json['Middle_C_X_Pos']), float(data_json['Middle_C_Y_Pos']), 1,
            float(data_json['Middle_D_X_Pos']), float(data_json['Middle_D_Y_Pos']), 1,
            # Ring, dataset 13-16
            float(data_json['Ring_A_X_Pos']), float(data_json['Ring_A_Y_Pos']), 1,
            float(data_json['Ring_B_X_Pos']), float(data_json['Ring_B_Y_Pos']), 1,
            float(data_json['Ring_C_X_Pos']), float(data_json['Ring_C_Y_Pos']), 1,
            float(data_json['Ring_D_X_Pos']), float(data_json['Ring_D_Y_Pos']), 1,
            # Pinky, dataset 17-20
            float(data_json['Pinky_A_X_Pos']), float(data_json['Pinky_A_Y_Pos']), 1,
            float(data_json['Pinky_B_X_Pos']), float(data_json['Pinky_B_Y_Pos']), 1,
            float(data_json['Pinky_C_X_Pos']), float(data_json['Pinky_C_Y_Pos']), 1,
            float(data_json['Pinky_D_X_Pos']), float(data_json['Pinky_D_Y_Pos']), 1,
           
            
        ]

    return key_points

In [119]:
def notPoseEstimate(k):
    if k[:5] == "Index" or k[:6] == "Middle" or k[:4] == "Ring" or k[:5] == "Pinky" or k[:5] == "Thumb" or k[:5] == "Wrist":
        return False

    return True

In [120]:
def make_dataset_json(input_csv, info_json, output_json=None):
    full_json = {}
    data_json = make_json(input_csv)

    #add info
    full_json['info'] = info_json
    full_json['liscense'] = ""
    full_json['images'] = []
    full_json['annotations'] = []
    for key in data_json:
        #image label processing
        image = {}
        image['id'] = key
        image['width'] = 576 
        image['height'] = 720
        image['file_name'] = getFileName(key)
        full_json['images'].append(image)

        #annotation processing
        annotation = {}
        annotation['id'] = key
        annotation['image_id'] = key
        for k in data_json[key]:
            if notPoseEstimate(k):
                annotation[k] = data_json[key][k]
        # annotation['iscrowd'] = 0
        # annotation['exposure'] = data_json[key]['Exposure']
        # annotation['rotation'] = data_json[key]['Rotation']
        

        # annotation['area'] = 0
        # annotation['bbox'] = [0,0,0,0]
        annotation['keypoints'] = getKeyPoints(data_json[key])
        full_json['annotations'].append(annotation)
        

    # return full_json
    if output_json is None:
        output_json = input_csv.replace('.csv', '.json')
    with open(output_json, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(full_json, indent=4))
    return output_json
    


In [121]:
csv_path = ROOT_DIR + '\\temp_data\\train10k.csv'
info_json = make_info_json('Hi510kDataset', '1.0', '2022', '2022/11/10')
data_set_path = make_dataset_json(csv_path, info_json)
print(data_set_path)


e:\Hi5-data\ViTPose\temp_data\train10k.json


keypoints are groups of 3,  x,y,confidence


format the COCO JSON I was sent cuz its ugly
`

In [124]:
def format_json(input_json, output_json = None):
    with open(input_json, 'r') as jsonf:
        data = json.load(jsonf)
    if output_json is None:
        output_json = input_json
    with open(output_json, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
    return output_json
format_json('E:\Hi5-data\ViTPose\\temp_data\person_keypoints_test3k.json')
format_json('E:\Hi5-data\ViTPose\\temp_data\person_keypoints_train10k.json')


'E:\\Hi5-data\\ViTPose\\temp_data\\person_keypoints_train10k.json'

In [2]:
import torch
torch.cuda.is_available()
# device = torch.device("cuda")



True

In [ ]:
"""
Onehand10k Format:
wrist
thumb1-4
forefinger1-4
middle_finger1-4
ring_finger1-4
pinky_finger1-4

Hi5 Format:
wrist
indexA-D
middleA-D
ringA-D
pinkyA-D
thumbA-D

for clarification: 
        The lowest number label is the base of the finger
        The highest number label is the tip of the finger
        matches proper ordrer of the data_checker process (see data_checker.py, getKeypoints())

    JSON format:
        X,Y, 2
        Wrist, IndexA-D, MiddleA-D, RingA-D, PinkyA-D, ThumbA-D

rewrite onehand10k to match hi510k style
"""